# Telecom Churn - ML Group Case Study

#### Business Problem Overview

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.
 
For many incumbent operators, retaining high profitable customers is the number one business goal.
 
To reduce customer churn, telecom companies need to predict which customers are at high risk of churn.
 
In this project, it is required to analyse customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn and identify the main indicators of churn.

##### Import Python libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
%matplotlib inline
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

## Data Preparation 

### 0. Loading the data

In [ ]:
churn = pd.read_csv("telecom_churn_data.csv")
churn.head()

In [ ]:
# Loading the data dictionary
data_dict = pd.read_excel("Data+Dictionary-+Telecom+Churn+Case+Study.xlsx")
data_dict.head()

### 1. Data Understanding

In [ ]:
churn.head()

In [ ]:
# Getting 5 random rows 
churn.sample(5)

In [ ]:
# Listing data types in the data
list(set(churn.dtypes.tolist()))

In [ ]:
churn.info()

In [ ]:
churn.shape
print("There are {} rows and {} columns in data".format(churn.shape[0],churn.shape[1]))

In [ ]:
churn.describe(include='all')

In [ ]:
# Looking at number of unique values in columns
churn.nunique()

In [ ]:
# Copying the data as backup
churn_original = churn.copy()

### 2. Data Cleansing

In [ ]:
# No of rows and columns at start
churn.shape

In [ ]:
#  First dropping the columns mobile_numer and circle_id as these wouldn't add any intelligence for modelling
churn = churn.drop(['mobile_number', 'circle_id'], axis=1)

In [ ]:
# After dropping the above columns
churn.shape

In [ ]:
# get the number of missing data points per column
missing_values_count = churn.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

In [ ]:
# Getting overall idea as how many total missing values do we have.
total_cells = np.product(churn.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

In [ ]:
def findMissingValues(data,range):
    return sum([1 for d in data if d>range])

# Finding the number of columns which are missing more than 70% values
missing_values = (churn.isnull().sum() / churn.isnull().count() * 100 ).sort_values(ascending = False)
findMissingValues(missing_values,70)

In [ ]:
# Find the columns with Object Types
obj_df = churn.select_dtypes(include='O')
obj_df.columns

In [ ]:
# Find the null values for Object data types
missing_na_obj = (obj_df.isnull().sum() / obj_df.isnull().count() * 100 ).sort_values(ascending = False)
missing_na_obj

In [ ]:
# Updating the null values in date of last rech for months (6-9) 
churn['date_of_last_rech_6'].fillna('6/30/2014',inplace=True)
churn['date_of_last_rech_7'].fillna('7/31/2014',inplace=True)
churn['date_of_last_rech_8'].fillna('8/31/2014',inplace=True)
churn['date_of_last_rech_9'].fillna('9/30/2014',inplace=True)

In [ ]:
# Find the columns with na values more than 70
missing_na_obj_cols = list(missing_na_obj.loc[missing_na_obj > 70].index)
missing_na_obj_cols

In [ ]:
# Dropping the above columns
churn = churn.drop(columns = missing_na_obj_cols,axis =1)

In [ ]:
# List the remaining date(object) type columns
date_cols_data = churn.select_dtypes(include='O')
date_cols_data.columns

In [ ]:
#  Now, Imputing missing values by modeling each feature with missing values as a function 
#  of other features in a round-robin fashion.

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Excluding the date types(Object)
df = churn.select_dtypes(exclude='O') 
# list(set(df.dtypes.tolist()))
imp = IterativeImputer(max_iter=5, verbose=0)
imp.fit(df)

In [ ]:
imputed_churn = imp.transform(df)
imputed_churn = pd.DataFrame(imputed_churn, columns=df.columns)

In [ ]:
# Find the values after imputing 
imported_churn_list = (imputed_churn.isnull().sum() / imputed_churn.isnull().count() * 100 ).sort_values(ascending = False)
print("Missing values after imputing = {}".format( sum(list(imported_churn_list.loc[imported_churn_list > 0].index))))

In [ ]:
# Combining back imputed and the date cols data
churn = pd.concat([imputed_churn, date_cols_data],axis=1)

In [ ]:
churn.shape

In [ ]:
# List the columns which are having constant value
non_obj = churn.select_dtypes(exclude='O') 
const_cols = non_obj.columns[non_obj.nunique() <= 1]
print ("There are {} columns which has 1 unique value. These columns are: \n{}".format(len(const_cols),const_cols))

In [ ]:
# Let's remove these columns as well, as all take constant values 
churn.drop(const_cols,axis=1,inplace=True)

In [ ]:
# Looking into date cols for null values
date_cols_data.isnull().sum()

In [ ]:
columns = ['last_date_of_month_7','last_date_of_month_8','last_date_of_month_9']
for col in columns:
    churn[col].fillna(churn[col].mode()[0], inplace=True)

In [ ]:
# Looking into date cols for null values after updating the values
date_cols_data.isnull().sum()

In [ ]:
churn.shape

In [ ]:
# Taking the data copy after cleansing
churn_cleansed = churn.copy()

### 3. Filter high-value customers & derive new features 

In [ ]:
churn.filter(regex='rech').columns

In [ ]:
# Deriving New Feature :Total no. of data recharge based on 2g + 3g 
for i in range(6,10):
    churn['total_rech_num_data_'+str(i)] = (churn['count_rech_2g_'+str(i)] +churn['count_rech_3g_'+str(i)]).astype(int)

In [ ]:
# Deriving New Feature : Total recharge amount for data for months(6-9)
for i in range(6,10):
    churn['total_rech_amt_data_'+str(i)] = churn['total_rech_num_data_'+str(i)] * churn['av_rech_amt_data_'+str(i)]

In [ ]:
# Deriving New Feature :Total rechange amount for voice(calling) and Data for months(6-9)
for i in range(6,10):
    churn['total_month_rech_'+str(i)] = churn['total_rech_amt_'+str(i)]+churn['total_rech_amt_data_'+str(i)]

In [ ]:
churn.filter(regex=('total_month_rech')).head()

In [ ]:
# Average of first 2 months of good phase (Months 6 & 7) 
avg_good_phase = (churn['total_month_rech_6'] +churn['total_month_rech_7'] )/2 

#### High-value customers definition:
-----------------------------------------------
##### Those who have recharged with an amount more than or equal to X, where X is the 70th percentile of the average recharge amount in the first two months (the good phase).

In [ ]:
hvc = churn[avg_good_phase >=  np.percentile(avg_good_phase , 70)]
hvc.reset_index(inplace=True , drop = True) 
print("High-value users {} with percentage {}% in data ".format(len(hvc), round(len(hvc)/churn.shape[0]*100),2))

In [ ]:
# Creating/Deriving some more new features based on the change between the 8th and avg of 6th and 7th month:

# Recharge amount Voice
hvc['total_rech_num_chng'] = hvc.total_rech_num_8 - ((hvc.total_rech_num_6 + hvc.total_rech_num_7)/2)
hvc['total_rech_amt_chng'] = hvc.total_rech_amt_8 - ((hvc.total_rech_amt_6 + hvc.total_rech_amt_7)/2)
hvc['max_rech_amt_chng'] = hvc.max_rech_amt_8 - ((hvc.max_rech_amt_6 + hvc.max_rech_amt_7)/2)

# Recharge amount Data
hvc['total_rech_data_chng'] = hvc.total_rech_data_8 - ((hvc.total_rech_data_6 + hvc.total_rech_data_7)/2)
hvc['max_rech_data_chng'] = hvc.max_rech_data_8 - ((hvc.max_rech_data_6 + hvc.max_rech_data_7)/2)
hvc['av_rech_amt_data_chng'] = hvc.av_rech_amt_data_8 - ((hvc.av_rech_amt_data_6 + hvc.av_rech_amt_data_7)/2)

# OnNet and Offnet usage 
hvc['onnet_mou_chng'] = hvc.onnet_mou_8 - ((hvc.onnet_mou_6 + hvc.onnet_mou_7)/2)
hvc['offnet_mou_chng'] = hvc.offnet_mou_8 - ((hvc.offnet_mou_6 + hvc.offnet_mou_7)/2)

# roaming icoming /outgoing
hvc['roam_ic_mou_chng'] = hvc.roam_ic_mou_8 - ((hvc.roam_ic_mou_6 + hvc.roam_ic_mou_7)/2)
hvc['roam_og_mou_chng'] = hvc.roam_og_mou_8 - ((hvc.roam_og_mou_6 + hvc.roam_og_mou_7)/2)

# Local calls usage
hvc['loc_og_mou_chng'] = hvc.loc_og_mou_8 - ((hvc.loc_og_mou_6 + hvc.loc_og_mou_7)/2)
hvc['loc_ic_mou_chng'] = hvc.loc_ic_mou_8 - ((hvc.loc_ic_mou_6 + hvc.loc_ic_mou_7)/2)

# STD icoming/outgoing
hvc['std_ic_mou_chng'] = hvc.std_ic_mou_8 - ((hvc.std_ic_mou_6 + hvc.std_ic_mou_7)/2)
hvc['std_og_mou_chng'] = hvc.std_og_mou_8 - ((hvc.std_og_mou_6 + hvc.std_og_mou_7)/2)

# ISD incoming/outgoing
hvc['isd_ic_mou_chng'] = hvc.isd_ic_mou_8 - ((hvc.isd_ic_mou_6 + hvc.isd_ic_mou_7)/2)
hvc['isd_og_mou_chng'] = hvc.isd_og_mou_8 - ((hvc.isd_og_mou_6 + hvc.isd_og_mou_7)/2)

# Special calls incoming/outgoing
hvc['spl_og_mou_chng'] = hvc.spl_og_mou_8 - ((hvc.spl_og_mou_6 + hvc.spl_og_mou_7)/2)
hvc['spl_ic_mou_chng'] = hvc.spl_ic_mou_8 - ((hvc.spl_ic_mou_6 + hvc.spl_ic_mou_7)/2)

# total incoming/outgoing
hvc['total_og_mou_chng'] = hvc.total_og_mou_8 - ((hvc.total_og_mou_6 + hvc.total_og_mou_7)/2)
hvc['total_ic_mou_chng'] = hvc.total_ic_mou_8 - ((hvc.total_ic_mou_6 + hvc.total_ic_mou_7)/2)

# Vol 2G/3G MB 
hvc['vol_2g_mb_chng'] = hvc.vol_2g_mb_8 - ((hvc.vol_2g_mb_6 + hvc.vol_2g_mb_7)/2)
hvc['vol_3g_mb_chng'] = hvc.vol_3g_mb_8 - ((hvc.vol_3g_mb_6 + hvc.vol_3g_mb_7)/2)

# ARPU for voice and 2G/3G 
hvc['arpu_chng'] = hvc.arpu_8 - ((hvc.arpu_6 + hvc.arpu_7)/2)
hvc['arpu_2g_chng'] = hvc.arpu_2g_8 - ((hvc.arpu_2g_6 + hvc.arpu_2g_7)/2)
hvc['arpu_3g_chng'] = hvc.arpu_3g_8 - ((hvc.arpu_3g_6 + hvc.arpu_3g_7)/2)


### 4.Tag churners and remove attributes of the churn phase

Now tag the churned customers (churn=1, else 0) based on the fourth month as follows: Those who have not made any calls (either incoming or outgoing) AND have not used mobile internet even once in the churn phase.
###### Attributes to tag churners are: total_ic_mou_9 , total_og_mou_9 , vol_2g_mb_9 , vol_3g_mb_9

In [ ]:
hvc['churn_flag'] = np.where(((hvc['total_ic_mou_9'] == 0.00) | (hvc['total_og_mou_9'] == 0.00)) & ((hvc['vol_2g_mb_9'] == 0.00)| (hvc['vol_3g_mb_9'] == 0.00)),1,0 )

In [ ]:
print("High-value Churn Percentage : {}%".format(round(len(hvc[hvc.churn_flag == 1])/hvc.shape[0] *100,2)))

###### This indicates that the dataset is highly imbalanced data set where hurn cases are ~ 10 % as opposed to to the no-churn cases with the majority of ~ 90%

Only when the class imbalance is high, e.g. 85%-90% points for one class and 10%-15% for the other, standard optimization criteria or performance measures may not be as effective and would need modification.

In [ ]:
# After tagging churners, remove all the attributes corresponding to the churn phase 
# (all attributes having ‘ _9’, etc. in their names).

hvc = hvc.drop(hvc.filter(regex='_9|sep', axis= 1).columns, axis = 1 )

In [ ]:
plt.figure(figsize=(10,5))
hvc['churn_flag'].value_counts().plot(kind = 'bar')
plt.ylabel('Churn counts')
plt.xlabel('Churn status')
plt.title('Churn status report',fontsize=12)

In [ ]:
hvc.filter(regex='date').columns

In [ ]:
# Converting the date columns to dattime types
for col in hvc.filter(regex='date'):
    hvc[col]=pd.to_datetime(hvc[col])

In [ ]:
# Days since recharge.
for i in range(6,9):
    hvc['days_since_recharge_'+str(i)] = (hvc["last_date_of_month_"+str(i)] - hvc["date_of_last_rech_"+str(i)]).dt.days

In [ ]:
# Dropping the date columns after changing to other columns as above
date_cols_to_drop = hvc.filter(regex='date').columns
hvc = hvc.drop(columns=date_cols_to_drop)

In [ ]:
hvc.head()

In [ ]:
#hvc[hvc.columns[~hvc.filter(regex='since').isnull().values.any()]]
#hvc[hvc.columns[~hvc.columns.isin(['churn_flag'])]]
(hvc.isnull().sum() / hvc.isnull().count() * 100 ).sort_values(ascending = False).head()

In [ ]:
hvc.shape

### 5.Outlier Treatment

In [ ]:
hvc.dtypes.value_counts()

In [ ]:
def drawBoxPlots(data):
    for col in data.columns:
        plt.figure()
        plt.clf() 
        sns.boxplot(data[col],palette="deep")
        plt.title(col)
        plt.show()

In [ ]:
drawBoxPlots(hvc)

In [ ]:
hvc.info()

In [ ]:
# def outlier_treatment(datacolumn):
#     sorted(datacolumn)
#     Q1,Q3 = np.percentile(datacolumn , [25,75])
#     IQR = Q3 — Q1
#     lower_range = Q1 — (1.5 * IQR)
#     upper_range = Q3 + (1.5 * IQR)
#     return lower_range,upper_range

# for col in hvc.columns:
#     lowerbound,upperbound = outlier_treatment(hvc.col)
#     outlier = hvc[(hvc.col < lowerbound) | (hvc.col > upperbound)]
#     hvc.drop( outlier.index , inplace=True)

In [ ]:
def cappingOutliers(data):
    data = data.clip_upper(data.quantile(0.90))    
    data = data.clip_lower(data.quantile(0.10))
    return data

In [ ]:
hvc=hvc.apply(lambda x: cappingOutliers(x))

In [ ]:
hvc.shape

In [ ]:
hvc_copy=hvc.copy()

## Modelling

In [ ]:
from sklearn.model_selection import train_test_split

X = hvc.drop(['churn_flag'], axis=1)
y = hvc['churn_flag']    

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7,test_size=0.3,random_state=100)

In [ ]:
#As there are large number of features,so we use PCA as dimensional reduction technique.
# Rescaling the features before PCA as it is sensitive to the scales of the features

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# fitting and transforming the scaler on train
X_train = pd.DataFrame(scaler.fit_transform(X_train),columns=X_train.columns)

# transforming the train using the already fit scaler
X_test = pd.DataFrame(scaler.fit_transform(X_test),columns=X_test.columns)

In [ ]:
print("Counts of churn before oversampling : {}".format(sum(y_train==1)))
print("Counts of no churn before oversampling : {}".format(sum(y_train==0)))
print("Churn rate before oversampling : {}% \n".format(round(sum(y_train==1)/len(y_train)*100,2)))

In [ ]:
#  Handling imbalance using imblearn(SMOTE)
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=100)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

In [ ]:
print("Counts of churn after oversampling : {}".format(sum(y_train_res==1)))
print("Counts of no churn after oversampling : {}".format(sum(y_train_res==0)))
print("Churn rate after oversampling : {}% \n".format(round(sum(y_train_res==1)/len(y_train_res)*100,2)))

In [ ]:
#Importing the PCA module
from sklearn.decomposition import PCA
pca = PCA(svd_solver='randomized', random_state=40)

In [ ]:
pca.fit(X_train_res)

In [ ]:
pca.components_

In [ ]:
colnames = list(X.columns)
pca_df = pd.DataFrame({'PC1':pca.components_[0],'PC2':pca.components_[1], 'Feature':colnames})
pca_df.head()

In [ ]:
%matplotlib inline
fig = plt.figure(figsize = (12,10))
plt.scatter(pca_df.PC1, pca_df.PC2)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
# for i, txt in enumerate(pca_df.Feature):
#     plt.annotate(txt, (pca_df.PC1[i],pca_df.PC2[i]))
plt.tight_layout()
plt.show()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
#Making the screeplot - plotting the cumulative variance against the number of components
%matplotlib inline
fig = plt.figure(figsize = (12,8))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
print("pca.explained_variance_ratio_: ",pca.explained_variance_ratio_.round(3)*100)

In [ ]:
print (pca.explained_variance_ratio_.cumsum())

In [ ]:
# The plot above and cumulative sum showed that ~ 60 components explains around 95% variance in the data set. 
# In order words, using PCA we have reduced 188 predictors to 60 without compromising on explained variance. 

from sklearn.decomposition import IncrementalPCA
pca_final = IncrementalPCA(n_components=60)

In [ ]:
X_train_pca = pca_final.fit_transform(X_train_res)
X_train_pca.shape

In [ ]:
#creating correlation matrix for the principal components
corrmat = np.corrcoef(X_train_pca.transpose())

In [ ]:
#plotting the correlation matrix
%matplotlib inline
plt.figure(figsize = (20,10))
sns.heatmap(corrmat,annot = True)

In [ ]:
# Correlation should be close to zero

corr_matrix_nod = corrmat - np.diagflat(corrmat.diagonal())
print("max corr:",corr_matrix_nod.max(), ", min corr: ", corr_matrix_nod.min(),)

In [ ]:
#Applying selected components to the test data
X_test_pca = pca_final.transform(X_test)
X_test_pca.shape

For the prediction of churn customers we will be fitting variety of models and select one which is the best predictor of churn. Models used are as below:
1. Logistic Regression
2. Decision Tree
3. Random Forest

In [ ]:
#Utility function to display the sensitivity , specificity , accuracy of the model 
def disp_Metrics(target_test , target_pred):
    confusion = confusion_matrix(target_test , target_pred)
    TP = confusion[1,1] # true positive 
    TN = confusion[0,0] # true negatives
    FP = confusion[0,1] # false positives
    FN = confusion[1,0] # false negatives 
    
    # Calculate false postive rate - predicting churn when customer does not have churned
    print("False postive rate - predicting churn when customer does not have churned :{}".format(round(FP/ float(TN+FP),2)))
    print("Positive predictive value :{}".format(round(TP/float(TP+FP),2)))
    print("Negative predictive value :{}".format(round(TN/float(TN+ FN),2)))
    print("Sensitivity :{}".format(round(TP/float(TP+FN),2)))
    print("Specificity :{}".format(round(TN/float(TN+FP),2)))
    print("Accuracy :{}".format(round(accuracy_score(target_test,target_pred),2)))

In [ ]:
# Utility Function to draw ROC
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None


###### Logistic Regression

Logistic Regression on principal components

In [ ]:
#Training the model on the train data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report

lr = LogisticRegression(class_weight='balanced')

In [ ]:
#Fit the algorithm on the data
model_pca = lr.fit(X_train_pca, y_train_res)

In [ ]:
#Predict the algorithm on the data
y_pred_prob = model_pca.predict_proba(X_test_pca)[:,1]

In [ ]:
# Converting y_test to dataframe
y_pred_df = pd.DataFrame(y_pred_prob)
y_pred_1 = y_pred_df.iloc[:,[0]]
y_pred_df.head()

y_test_df = pd.DataFrame(y_test)
y_test_df.reset_index(drop=True, inplace=True)

y_pred_final = pd.concat([y_test_df,y_pred_1],axis=1)
y_pred_final= y_pred_final.rename(columns={ 0 : 'churn_prob'})

y_pred_final['predicted'] = y_pred_final.churn_prob.map( lambda x: 1 if x > 0.5 else 0)
y_pred_final.head()

In [ ]:
# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_pred_final[i]= y_pred_final.churn_prob.map( lambda x: 1 if x > i else 0)
y_pred_final.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
from sklearn.metrics import confusion_matrix
num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix( y_pred_final.churn_flag, y_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    sensi = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    speci = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])

In [ ]:
# From above optimal cutoff diagram taking prob as 0.5
cutoff = 0.5
y_pred= [1 if proba > cutoff else 0 for proba in y_pred_prob]
print('AUC:', round(roc_auc_score(y_test, y_pred),2))

In [ ]:
# Displaying metrics
disp_Metrics(y_test,y_pred)

In [ ]:
# drawing ROC
draw_roc(y_test,y_pred)

###### Decision Tree

In [ ]:
# Importing decision tree classifier from sklearn library
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(class_weight='balanced',
                             max_features='auto',
                             min_samples_split=100,
                             min_samples_leaf=100,
                             max_depth=6,
                             random_state=10)
dt.fit( X_train_pca, y_train_res)

In [ ]:
# Making predictions
y_pred = dt.predict(X_test_pca)

In [ ]:
df = pd.DataFrame(confusion_matrix(y_test, y_pred), range(2),range(2))
sns.set(font_scale=1.2)
sns.heatmap(df, annot=True,annot_kws={"size": 12})
plt.show()

In [ ]:
disp_Metrics(y_test , y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

##### Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Create the parameter grid 
param_grid = {
    'max_depth': range(5, 15, 5),
    'min_samples_leaf': range(50, 150, 50),
    'min_samples_split': range(50, 150, 50),
    'criterion': ["entropy", "gini"]
}

n_folds = 5
# Instantiate the grid search model
dtree = DecisionTreeClassifier(class_weight='balanced')
grid_search = GridSearchCV(estimator = dtree, param_grid = param_grid, 
                          cv = n_folds, verbose = 1,n_jobs=-1,scoring='recall')

# Fit the grid search to the data
grid_search.fit(X_train_pca, y_train_res)

In [ ]:
print(grid_search.best_params_)

In [ ]:
# scores of GridSearch CV
scores = grid_search.cv_results_
pd.DataFrame(scores).head()

In [ ]:
# Instantiate the grid search model
dtree = DecisionTreeClassifier(class_weight='balanced',max_depth=10,min_samples_leaf=50,min_samples_split=50)

# Fit the grid search to the data
dtree.fit(X_train_pca, y_train_res)

In [ ]:
# Making predictions
y_pred = dtree.predict(X_test_pca)

In [ ]:
#display the heatmap
df = pd.DataFrame(confusion_matrix(y_test, y_pred), range(2),range(2))
sns.set(font_scale=1.2)
sns.heatmap(df, annot=True,annot_kws={"size": 12})
plt.show()

In [ ]:
#confusion matrix
disp_Metrics(y_test,y_pred)

In [ ]:
#roc_auc_score
pred_test = dtree.predict_proba(X_test_pca)[:,1]
print('roc_auc_score:', round(roc_auc_score(y_test, pred_test),2))

###### Random Forest on PCA dataset:


In [ ]:
# Importing random forest classifier from sklearn library
from sklearn.ensemble import RandomForestClassifier

# Running the random forest with default parameters.
rfc = RandomForestClassifier()

In [ ]:
#fit the model
rfc.fit(X_train_pca,y_train_res)

In [ ]:
#predict on test data
y_pred = rfc.predict(X_test_pca)

In [ ]:
df = pd.DataFrame(confusion_matrix(y_test, y_pred), range(2),range(2))
sns.set(font_scale=1.2)
sns.heatmap(df, annot=True,annot_kws={"size": 12})
plt.show()

In [ ]:
#confusion matrix
disp_Metrics(y_test,y_pred)

In [ ]:
pred_test = rfc.predict_proba(X_test_pca)[:,1]
round(metrics.roc_auc_score(y_test, pred_test),2)

###### Hyperparameter Tuning

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [4,8,10],
    'min_samples_leaf': range(100, 400, 200),
    'min_samples_split': range(200, 500, 200),
    'n_estimators': [100,200, 300], 
    'max_features': [5, 10]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1,verbose = 1,scoring='recall')

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train_pca, y_train_res)

In [ ]:
#Recall
print(grid_search.best_score_ )
#optimal accuracy
print(grid_search.best_params_)

In [ ]:
# model with the best hyperparameters
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=5,
                             min_samples_leaf=100, 
                             min_samples_split=200,
                             max_features=5,
                             n_estimators=100
                             )

In [ ]:
# fit the algorithm on the data
rfc.fit(X_train_pca,y_train_res)

In [ ]:
# Making predictions
y_pred = rfc.predict(X_test_pca)
y_pred[:5]

In [ ]:
df = pd.DataFrame(confusion_matrix(y_test, y_pred), range(2),range(2))
sns.set(font_scale=1.2)
sns.heatmap(df, annot=True,annot_kws={"size": 12})
plt.show()

In [ ]:
#confusion matrix
disp_Metrics(y_test,y_pred)

In [ ]:
pred_probs_test = rfc.predict_proba(X_test_pca)[:,1]
metrics.roc_auc_score(y_test, pred_probs_test)

Overall, the Logistic Regression model with probability performs best. It achieved the best recall accuracy of 83% for test data.

## Tree based model to identify the important predictor variables.

In [ ]:
# Importing random forest classifier from sklearn library
from sklearn.ensemble import RandomForestClassifier

# Running the random forest with default parameters.
rfc = RandomForestClassifier(class_weight={0:0.11,1:0.89})

In [ ]:
# fit
rfc.fit(X_train,y_train)

In [ ]:
# Making predictions
y_pred = rfc.predict(X_test)

In [ ]:
df = pd.DataFrame(confusion_matrix(y_test, y_pred), range(2),range(2))
sns.set(font_scale=1.2)
sns.heatmap(df, annot=True,annot_kws={"size": 12})
plt.show()

In [ ]:
#confusion matrix
disp_Metrics(y_test,y_pred)

In [ ]:
pred_probs_test = rfc.predict_proba(X_test)[:,1]
metrics.roc_auc_score(y_test, pred_probs_test)

The sensitivity is poor for the default model. Let's try tuning the hypeparameters to improve the model performance.

# Hyperparameter Tuning 

In [ ]:
param_grid = {
    'max_depth': [4,8,10],
    'min_samples_leaf': range(100, 300, 100),
    'min_samples_split': range(200, 400, 100),
    'n_estimators': [100,200,300,500], 
    'max_features': [5,10]
}
# Create a Random Forest based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1,verbose = 1,scoring='recall')

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
# model with the best hyperparameters
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=10,
                             min_samples_leaf=100, 
                             min_samples_split=300,
                             max_features=10,
                             n_estimators=500,
                             class_weight={0:0.11,1:0.89})

In [ ]:
# fit
rfc.fit(X_train,y_train)

In [ ]:
# Making predictions
y_pred = rfc.predict(X_test)
y_pred[:5]

In [ ]:
df = pd.DataFrame(confusion_matrix(y_test, y_pred), range(2),range(2))
sns.set(font_scale=1.2)
sns.heatmap(df, annot = True, annot_kws = {"size": 12})
plt.show()

In [ ]:
#confusion matrix
disp_Metrics(y_test,y_pred)

In [ ]:
pred_probs_test = rfc.predict_proba(X_test)[:,1]
metrics.roc_auc_score(y_test, pred_probs_test)

Now the model metrics are quite satisfactory. Let's now proceed with identifying important predictor variables.

## Feature Importance

In [ ]:
X_train.columns

In [ ]:
# Print the name and gini importance of each feature
features = X_train.columns
for feature in zip(features, rfc.feature_importances_):
    print(feature)

In [ ]:
feature_df = pd.concat([pd.Series(features),pd.Series(rfc.feature_importances_)],axis=1).rename(columns={0:'Features',1:'Weight'})
feature_df.head(10)

In [ ]:
# Top 20 features
top_20 = feature_df.sort_values('Weight',ascending=False).head(20)
top_20

In [ ]:
plt.figure(figsize=(20,6))
ax = sns.barplot(data = top_20, x='Features', y='Weight',edgecolor=sns.color_palette('dark',5))
ax.set_title("Top 20 Churn Predictors")
plt.xticks(rotation=90)
plt.show()

# Exploratory Analysis

In [ ]:
def plot_byChurn(col,colList):
    # per month churn vs Non-Churn
    fig, ax = plt.subplots(figsize=(7,4))
    plt.plot(hvc.groupby('churn_flag')[colList].mean().T)
    ax.set_xticklabels(['Jun','Jul','Aug','Sep'])
    ## Add legend
    plt.legend(['No Churn', 'Churn'])
    # Add titles
    plt.title(col, loc='left', fontsize=14, fontweight=0, color='blue')
    plt.xlabel("Month")
    plt.show()

In [ ]:
def pair_plot(data):
    sns.set(style="ticks", color_codes=True)
    hue = 'churn_flag'
    sns.pairplot(data,hue,vars=data.iloc[:,:3],size=3, palette="husl",markers=["o", "s"])

In [ ]:
ic_col = ['total_ic_mou_6','total_ic_mou_7','total_ic_mou_8']
plot_byChurn("total_ic_mou",ic_col )

In [ ]:
total_og_mou_col =  ['total_og_mou_6' ,  'total_og_mou_7' ,  'total_og_mou_8']
plot_byChurn( total_og_mou_col )

In [ ]:
loc_og_mou_col =  ['loc_og_mou_6' ,  'loc_og_mou_7' ,  'loc_og_mou_8']
plot_byChurn("loc_og_mou", loc_og_mou_col )

In [ ]:
total_rech_amt_col = ['total_rech_amt_6','total_rech_amt_7','total_rech_amt_8']
plot_byChurn("total_rech_amt",total_rech_amt_col)

In [ ]:
roam_og_mou_col = ['roam_og_mou_6', 'roam_og_mou_7','roam_og_mou_8']
plot_byChurn("roam_og_mou",roam_og_mou_col)

In [ ]:
roam_ic_mou_col = ['roam_ic_mou_6', 'roam_ic_mou_7','roam_ic_mou_8']
plot_byChurn("roam_ic_mou_6",roam_ic_mou_col)

In [ ]:
arpu_col = ['arpu_6', 'arpu_7','arpu_8']
plot_byChurn("arpu",arpu_col)

In [ ]:
plt.figure(figsize=(8,4))
hvc['churn_flag'].value_counts().plot(kind = 'bar')
plt.ylabel('Count')
plt.xlabel('Churn')
plt.title('Churn Distribution',fontsize=14)

In [ ]:
#Average Revenue per user 
data=hvc[['arpu_6' , 'arpu_7' , 'arpu_8','churn_flag']]
pair_plot(data)

In [ ]:
# Usage within same network
data = hvc[['onnet_mou_6' , 'onnet_mou_7' , 'onnet_mou_8' , 'churn_flag']]
pair_plot(data)

In [ ]:
# Usage outside of the operator network
data=hvc[['offnet_mou_6' ,'offnet_mou_7' ,'offnet_mou_8' ,'churn_flag']]
pair_plot(data)

In [ ]:
# Local outgoing Calls
data= hvc[['loc_og_mou_6','loc_og_mou_7','loc_og_mou_8','churn_flag']]
pair_plot(data)

In [ ]:
# STD outgoing calls
data= hvc[['std_og_mou_6','std_og_mou_7','std_og_mou_8','churn_flag']]
pair_plot(data)

In [ ]:
# ISD outgoing calls
data=hvc[['isd_og_mou_6','isd_og_mou_7','isd_og_mou_8','churn_flag']]
pair_plot(data)

In [ ]:
# Total outgoing calls
data=hvc[['total_og_mou_6','total_og_mou_7','total_og_mou_8','churn_flag']]
pair_plot(data)

In [ ]:
# Roaming outgoing Calls
data= hvc[['roam_og_mou_6','roam_og_mou_7','roam_og_mou_8','churn_flag']]
pair_plot(data)

In [ ]:
# Incoming Roaming Calls
data= hvc[['roam_ic_mou_6','roam_ic_mou_7','roam_ic_mou_8','churn_flag']]
pair_plot(data)

In [ ]:
# Local incoming calls
data=hvc[['loc_ic_mou_6','loc_ic_mou_7','loc_ic_mou_8','churn_flag']]
pair_plot(data)

In [ ]:
# STD incoming calls
data=hvc[['std_ic_mou_6','std_ic_mou_7','std_ic_mou_8','churn_flag']]
pair_plot(data)

In [ ]:
# Total incoming calls
data=hvc[['total_ic_mou_6','total_ic_mou_7','total_ic_mou_8','churn_flag']]
pair_plot(data)

In [ ]:
# Total recharge numbers for voice
data=hvc[['total_rech_num_6','total_rech_num_7','total_rech_num_8','churn_flag']]
pair_plot(data)

In [ ]:
# Total recharge amount for voice
data=hvc[['total_rech_amt_6','total_rech_amt_7','total_rech_amt_8','churn_flag']]
pair_plot(data)

In [ ]:
# 2G data usage
data= hvc[['vol_2g_mb_6','vol_2g_mb_7','vol_2g_mb_8','churn_flag']]
pair_plot(data)

In [ ]:
# 3G data usage
data= hvc[['vol_3g_mb_6','vol_3g_mb_7','vol_3g_mb_8','churn_flag']]
pair_plot(data)

In [ ]:
# Volume based
data= hvc[['jun_vbc_3g','jul_vbc_3g','aug_vbc_3g','churn_flag']]
pair_plot(data)

#### Business Indicators/Recommendations/strategies

It is seen and clear from the Feature importance section, that action phase(Month 8th) has most significant impact on the customer churn of high value customers, and thus focussing on these features forms the clear indication for the churn.Overall, a reduction in any of these indicator KPIs suggest that customer is not fulling partaking in the services offered and thus may potentially churn in the near future.

It is also understood from above that incoming & outgoing calls(both roaming and local) are key for identifying churn customers, and this is especially important during the action phase in comparison to the 6th and 7th month(Good phase), since these are observed to be reduced.

The other important feature(factor) is the recharge amount which also showed a dip in the action phase(8th month).The average revenue per user has also observed decline when compared with 7th month, so ARPU is also another important consideration business should be looking into to avoid churn.

Overall business sholud atleast be closely monitoring these important features on more short duration.